# Buy and Hold
1. 구매 후 일정기간 지나면 판매

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../csv/AAPL.csv", index_col='Date')
df

In [ ]:
# 결측치와 무한대의 값이 존재하는지 확인
df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis='columns')]

In [ ]:
df.dropna(axis=0)

In [ ]:
# df에 있는 수정종가 부분만 추출하여 데이터프레임의 형태로
# case1
df[['Adj Close']]
# case2
price_df = df.loc[:, ['Adj Close']]

In [ ]:
# 수정종가를 그래프로 표시 
price_df.plot()

In [ ]:
# 파생 변수 생성(일별 수익율)
# 일별 수익율 =   (오늘의 종가 - 전날의 종가) / 전날의 종가 
# case1
price_df['Adj Close'].diff() / price_df['Adj Close'].shift()
# case2
price_df['daily_rtn'] = price_df['Adj Close'].pct_change()

In [ ]:
price_df.head(5)

In [30]:
# 일별 수익율을 가지고 총 수익율 구하려면?

test_df = price_df.iloc[500:600]

In [34]:
rtn = 1

for i in test_df.index:
    rtn = rtn * (1+ test_df.loc[i, 'daily_rtn'])
print(rtn)
    

1.4907733209842349


In [37]:
(1 + test_df['daily_rtn']).cumprod()

Date
1982-12-08    0.977858
1982-12-09    0.929889
1982-12-10    0.863469
1982-12-13    0.845017
1982-12-14    0.837637
                ...   
1983-04-25    1.435422
1983-04-26    1.476012
1983-04-27    1.461253
1983-04-28    1.476012
1983-04-29    1.490773
Name: daily_rtn, Length: 100, dtype: float64

In [41]:
# 종가를 기준으로 수익율 계산

price_df['st_rtn'] = (1 + price_df['daily_rtn']).cumprod()

In [46]:
price_df.tail(1)

,Adj Close,daily_rtn,st_rtn
Date,,,
2019-06-24,199.169998,0.001962,485.159242


In [45]:
# 첫날의 종가와 마지막 날의 종가
first = price_df.iloc[0, 0]
last = price_df.iloc[-1, 0]
print(first, last)

0.410525 199.169998


In [47]:
print(last / first)

485.15924243346933


# Buy and Hold 함수로 생성 
1. 매개변수는 1개(데이터프레임)
2. 결측치, 무한대를 제거 
3. 수정종가만 있는 데이터프레임으로 변경
4. 일별 수익율 파생변수 생성
5. 누적 수익율 파생변수를 생성
6. 데이터프레임을 리턴


In [48]:
df = pd.read_csv("../csv/AAPL.csv")

In [60]:
# 함수에 기준이되는 컬럼을 매개변수 생성
# 매개변수 2개 구매 시기와 판매 시기

def buyandhold(df, column, start, end):
    # 결측치, 무한대 제외
    df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis='columns')]
    # 수정 종가만 있는 데이터프레임으로 변경
    df = df[[column]] 
    # 일별 수익율 변수 생성
    df['daily_rtn'] = df[column].pct_change()
    # 누적 수입율 변수 생성
    df['st_rtn'] = (1 + df['daily_rtn']).cumprod()
    # 데이터프레임을 리턴
    return df


In [ ]:
buyandhold(df)

In [58]:
df = pd.read_csv("../csv/AMZN.csv", index_col='Date')

In [63]:
df.index

Index(['1997-05-15', '1997-05-16', '1997-05-19', '1997-05-20', '1997-05-21',
       '1997-05-22', '1997-05-23', '1997-05-27', '1997-05-28', '1997-05-29',
       ...
       '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14', '2019-06-17',
       '2019-06-18', '2019-06-19', '2019-06-20', '2019-06-21', '2019-06-24'],
      dtype='object', name='Date', length=5563)

In [96]:
df.index = pd.to_datetime(df.index, format="%Y-%m-%d")

In [115]:
from datetime import datetime

In [118]:
start = "20080101"
end = "20150101"
start = datetime.strptime(start, '%Y%m%d').isoformat()
end = datetime.strptime(end, '%Y%m%d').isoformat()
start

'2008-01-01T00:00:00'

In [119]:
df.loc[start:end]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-02,78.260002,78.260002,76.910004,77.360001,54.651798,2723300
2008-01-03,78.050003,78.050003,77.129997,77.459999,54.722462,281400
2008-01-04,77.739998,77.739998,77.480003,77.550003,54.786034,231100
2008-01-07,77.699997,77.699997,77.370003,77.570000,54.800144,186300
2008-01-08,77.889999,77.889999,77.360001,77.650002,54.856682,144000
...,...,...,...,...,...,...
2014-12-24,81.989998,82.040001,81.849998,82.040001,73.073853,2280100
2014-12-26,82.110001,82.190002,82.050003,82.110001,73.136253,1300800
2014-12-29,82.239998,82.349998,82.160004,82.250000,73.260918,2730600


In [120]:
import buyandhold
# 라이브러리 리로드 
import imp
imp.reload(buyandhold)

<module 'buyandhold' from 'c:\\Users\\moons\\Documents\\GitHub\\ubion_7\\230412\\buyandhold.py'>

In [121]:
df = pd.read_csv("../csv/BND.csv", index_col='Date')

In [122]:
buyandhold.bnh(df, 'Adj Close', '20100101', '20150101')

,Adj Close,daily_rtn,st_rtn
Date,,,
2010-01-04,60.611969,NaN,NaN
2010-01-05,60.789135,0.002923,1.002923
2010-01-06,60.766037,-0.000380,1.002542
2010-01-07,60.719822,-0.000761,1.001779
2010-01-08,60.781410,0.001014,1.002796
...,...,...,...
2014-12-24,73.073853,0.000366,1.205601
2014-12-26,73.136253,0.000854,1.206631
2014-12-29,73.260918,0.001705,1.208687
